In [20]:
import pandas as pd
import os

def create_definitive_master_dataset():
    """
    Loads all historical data, cleans and standardizes key columns first,
    then merges them into a single, reliable master DataFrame.
    """
    print("Starting the definitive data merging process...")
    data_path = '../data/'

    try:
        # 1. Load Your Four Historical Datasets
        df_league_standings = pd.read_csv(os.path.join(data_path, 'combined_league_standings_2011-2025.csv'))
        df_league_players = pd.read_csv(os.path.join(data_path, 'combined_player_stats_2011-2025_CLEAN.csv'))
        df_ucl_players = pd.read_csv(os.path.join(data_path, 'all_ucl_player_stats_2011-2025_CLEAN.csv'))
        df_ucl_teams = pd.read_csv(os.path.join(data_path, 'all_ucl_team_performance_and_progress.csv'))
        
        print("All four historical datasets loaded successfully.")

    except FileNotFoundError as e:
        print(f"\nERROR: A file was not found. Please check your filenames. Missing file: {e.filename}")
        return

    # 2. PRE-MERGE CLEANING AND STANDARDIZATION
    print("\nCleaning and standardizing names for a reliable merge...")
    
    name_replacements = {
        'Paris S-G': 'Paris Saint-Germain', 'Inter': 'Internazionale', 'Manchester Utd': 'Manchester United'
    }
    
    all_dfs = [df_league_standings, df_league_players, df_ucl_players, df_ucl_teams]
    for df in all_dfs:
        df.columns = df.columns.str.strip()
        if 'Player' in df.columns:
            df['Player'] = df['Player'].str.strip()
        if 'Squad' in df.columns:
            df['Squad'] = df['Squad'].str.strip()
            df['Squad'] = df['Squad'].replace(name_replacements)
        if 'Season' in df.columns:
            df['Season'] = df['Season'].astype(str).str.strip()

    # 3. PERFORM THE MERGES
    master_df = pd.merge(df_league_players, df_league_standings, on=['Squad', 'League', 'Season'], how='left', suffixes=('_player', '_team'))
    
    ucl_stats_to_add = df_ucl_players[['Player', 'Squad', 'Gls', 'Ast', 'Min', 'Season']]
    master_df = pd.merge(master_df, ucl_stats_to_add, on=['Player', 'Squad', 'Season'], how='left', suffixes=('_league', '_ucl'))
    
    # This specifically selects ONLY the 'UCL_progress' column (and keys) to add
    ucl_progress_to_add = df_ucl_teams[['Squad', 'Season', 'UCL_progress']]
    master_df = pd.merge(master_df, ucl_progress_to_add, on=['Squad', 'Season'], how='left')
    
    # 4. FINAL CLEANUP
    ucl_cols = ['Gls_ucl', 'Ast_ucl', 'Min_ucl']
    for col in ucl_cols:
        if col in master_df.columns:
            master_df[col] = master_df[col].fillna(0)
    
    master_df['UCL_progress'] = master_df['UCL_progress'].fillna('Did Not Qualify')
    
    master_df = master_df.loc[:,~master_df.columns.duplicated()]

    print("\nAll merges and cleanup complete.")
    return master_df

# --- Main Execution ---
final_master_dataset = create_definitive_master_dataset()

if final_master_dataset is not None:
    output_path = os.path.join("../data", "master_dataset_2011-2025.csv")
    final_master_dataset.to_csv(output_path, index=False)

    print(f"\n--- SUCCESS! ---")
    print(f"Final master dataset created and saved to: {output_path}")
    display(final_master_dataset.head())
else:
    print("\nMerging process failed.")

Starting the definitive data merging process...
All four historical datasets loaded successfully.

Cleaning and standardizing names for a reliable merge...


C:\Users\akash\AppData\Local\Temp\ipykernel_15116\3033831686.py:15: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  df_league_players = pd.read_csv(os.path.join(data_path, 'combined_player_stats_2011-2025_CLEAN.csv'))



All merges and cleanup complete.

--- SUCCESS! ---
Final master dataset created and saved to: ../data\master_dataset_2011-2025.csv


,Rk_player,Player,Nation,Pos,Squad,Age,Born,MP_player,Starts,Min_league,...,Notes,xG_team,xGA,xGD,xGD/90,Last 5,Gls_ucl,Ast_ucl,Min_ucl,UCL_progress
0,1.0,Mohammed Abdellaoue,NOR,FW,Hannover 96,24.0,1985.0,26.0,26.0,0.0,...,→ Europa League via league finish,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Did Not Qualify
1,2.0,Yacine Abdessadki,MAR,MF,Freiburg,29.0,1981.0,21.0,20.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Did Not Qualify
2,3.0,Mathias Abel,GER,DF,Kaiserslautern,29.0,1981.0,19.0,19.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Did Not Qualify
3,4.0,René Adler,GER,GK,Leverkusen,25.0,1985.0,32.0,32.0,0.0,...,→ Champions League via league finish,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Did Not Qualify
4,5.0,David Alaba,AUT,"DF,MF",Hoffenheim,18.0,1992.0,17.0,17.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,Did Not Qualify


In [23]:
import pandas as pd
import os

def display_column_names():
    """
    Loads each of the four main data files and prints their column names.
    """
    print("--- Displaying Column Names for Each CSV File ---")
    
    # Path to your data folder, relative to the notebook's location
    data_path = '../data/'
    
    # List of the four files you want to inspect
    files_to_check = [
        'combined_league_standings_2011-2025.csv',
        'combined_player_stats_2011-2025_CLEAN.csv',
        'all_ucl_player_stats_2011-2025_CLEAN.csv',
        'all_ucl_team_performance_and_progress.csv'
    ]
    
    for filename in files_to_check:
        try:
            file_path = os.path.join(data_path, filename)
            
            # Read the CSV file
            df = pd.read_csv(file_path)
            
            # Print the filename and its list of columns
            print(f"\n--- Columns in: {filename} ---")
            print(df.columns.tolist())
            
        except FileNotFoundError:
            print(f"\n--- ERROR: Could not find file: {filename} ---")
        except Exception as e:
            print(f"\n--- An error occurred with {filename}: {e} ---")

# --- Main Execution ---
display_column_names()

--- Displaying Column Names for Each CSV File ---

--- Columns in: combined_league_standings_2011-2025.csv ---
['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'Pts/MP', 'Top Team Scorer', 'Goalkeeper', 'League', 'Season', 'xG', 'xGA', 'xGD', 'xGD/90', 'Last 5']

--- Columns in: combined_player_stats_2011-2025_CLEAN.csv ---
['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'Gls_per_90', 'Ast_per_90', 'G+A_per_90', 'G-PK_per_90', 'G+A-PK_per_90', 'Season', 'League', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'xG_per_90', 'xAG_per_90', 'xG+xAG_per_90', 'npxG_per_90', 'npxG+xAG_per_90']

--- Columns in: all_ucl_player_stats_2011-2025_CLEAN.csv ---
['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Age', 'Born', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'Gls_per_90', 'Ast_per_90', 'G+A_per_90', 'G-PK_per_90', 'G+A-PK_per_90', 

In [27]:
import pandas as pd
import os

def clean_ucl_teams_file():
    """
    Loads the UCL teams file, finds and removes all 'Unnamed' columns,
    and saves the clean version, overwriting the original.
    """
    print("Starting the cleaning process for 'all_ucl_team_performance_and_progress.csv'...")
    
    # Define the path to the file
    file_path = '../data/all_ucl_team_performance_and_progress.csv'
    
    try:
        # Load the file
        df = pd.read_csv(file_path, low_memory=False)
        print("File loaded successfully.")

    except FileNotFoundError:
        print(f"ERROR: The file was not found at {file_path}")
        return

    # --- Find and Remove 'Unnamed' columns from the entire DataFrame ---
    # 1. Get a list of all columns that contain 'Unnamed'
    unnamed_cols = [col for col in df.columns if 'Unnamed' in col]
    
    if unnamed_cols:
        # 2. Drop these columns
        df.drop(columns=unnamed_cols, inplace=True)
        print(f"Successfully found and removed the following useless columns: {unnamed_cols}")
    else:
        print("No 'Unnamed' columns were found to remove.")

    # --- Save the Cleaned File ---
    # This overwrites your old file with the corrected version
    df.to_csv(file_path, index=False)
    
    print(f"\n--- SUCCESS! ---")
    print(f"The file '{os.path.basename(file_path)}' has been permanently cleaned and saved.")
    
    # Display the new, clean column names to confirm
    print("\nHere are the final, clean column names:")
    print(df.columns.tolist())

# --- Run the cleaning process ---
clean_ucl_teams_file()

Starting the cleaning process for 'all_ucl_team_performance_and_progress.csv'...
File loaded successfully.
No 'Unnamed' columns were found to remove.

--- SUCCESS! ---
The file 'all_ucl_team_performance_and_progress.csv' has been permanently cleaned and saved.

Here are the final, clean column names:
['Rk', 'Squad', 'MP', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'Pts', 'Attendance', 'Top Team Scorer', 'Goalkeeper', 'Notes', 'Season']


In [32]:
import pandas as pd
import os

def create_and_audit_master_dataset():
    """
    Loads all historical data, performs a robust pre-merge cleaning, merges the data,
    and provides a clear audit of the merge results.
    """
    print("Starting the definitive data merging process...")
    data_path = '../data/'

    try:
        # --- 1. Load All Four Historical Datasets ---
        df_league_standings = pd.read_csv(os.path.join(data_path, 'combined_league_standings_2011-2025.csv'))
        df_league_players = pd.read_csv(os.path.join(data_path, 'combined_player_stats_2011-2025_CLEAN.csv'))
        df_ucl_players = pd.read_csv(os.path.join(data_path, 'all_ucl_player_stats_2011-2025_CLEAN.csv'))
        df_ucl_teams = pd.read_csv(os.path.join(data_path, 'all_ucl_team_performance_and_progress.csv'))
        print("All four historical datasets loaded successfully.")

    except FileNotFoundError as e:
        print(f"\nERROR: A file was not found. Please check your filenames. Missing file: {e.filename}")
        return

    # --- 2. PRE-MERGE CLEANING AND STANDARDIZATION ---
    print("\nStep 1/5: Cleaning and standardizing all data for a perfect merge...")
    
    all_dfs = [df_league_standings, df_league_players, df_ucl_players, df_ucl_teams]
    for df in all_dfs:
        df.columns = df.columns.str.strip()
        if 'Player' in df.columns: df['Player'] = df['Player'].str.strip()
        if 'Squad' in df.columns: df['Squad'] = df['Squad'].str.strip()
        if 'Season' in df.columns: df['Season'] = df['Season'].astype(str).str.strip()
    
    print("Data cleaning and standardization complete.")

    # --- 3. EXPLICIT CHECK FOR 'UCL_progress' COLUMN ---
    print("\nStep 2/5: Verifying 'UCL_progress' column exists...")
    if 'UCL_progress' not in df_ucl_teams.columns:
        print("\n--- FATAL ERROR ---")
        print("The column 'UCL_progress' was NOT FOUND in your 'all_ucl_team_performance_and_progress.csv' file.")
        print("Please rename the column containing the progress data (e.g., 'Rk' or 'Notes') to 'UCL_progress' in your CSV file and try again.")
        return
    print("'UCL_progress' column successfully found.")

    # --- 4. PERFORM THE MERGES ---
    print("\nStep 3/5: Merging all datasets...")
    master_df = pd.merge(df_league_players, df_league_standings, on=['Squad', 'League', 'Season'], how='left', suffixes=('_player', '_team'))
    ucl_stats_to_add = df_ucl_players[['Player', 'Squad', 'Gls', 'Ast', 'Min', 'Season']]
    master_df = pd.merge(master_df, ucl_stats_to_add, on=['Player', 'Squad', 'Season'], how='left', suffixes=('_league', '_ucl'))
    ucl_progress_to_add = df_ucl_teams[['Squad', 'Season', 'UCL_progress']]
    master_df = pd.merge(master_df, ucl_progress_to_add, on=['Squad', 'Season'], how='left')
    print("All merges complete.")

    # --- 5. FINAL CLEANUP ---
    print("\nStep 4/5: Performing final cleanup...")
    ucl_cols = ['Gls_ucl', 'Ast_ucl', 'Min_ucl']
    for col in ucl_cols:
        if col in master_df.columns:
            master_df[col] = master_df[col].fillna(0)
    master_df['UCL_progress'] = master_df['UCL_progress'].fillna('Did Not Qualify')
    master_df = master_df.loc[:,~master_df.columns.duplicated()]
    print("Cleanup complete.")
    
    # --- 6. SAVE THE DATASET ---
    output_path = os.path.join(data_path, "master_dataset_2011-2025.csv")
    master_df.to_csv(output_path, index=False)
    print(f"\nStep 5/5: Master dataset created and saved to: {output_path}")

    # --- 7. MERGE AUDIT AND EXPLANATION ---
    print("\n\n--- MERGE AUDIT & EXPLANATION ---")
    print("\nTo verify the merge, let's check some famous players known for UCL success:")
    
    # Check a few specific examples
    audit_players = master_df[master_df['Player'].isin(['Lionel Messi', 'Cristiano Ronaldo', 'Karim Benzema'])]
    successful_examples = audit_players[audit_players['UCL_progress'] != 'Did Not Qualify']

    if not successful_examples.empty:
        print("\n✅ AUDIT PASSED: The merge appears successful. Here are some examples where UCL data was correctly matched:")
        display(successful_examples[['Season', 'Player', 'Squad', 'Gls_ucl', 'UCL_progress']].head(10))
    else:
        print("\n❌ AUDIT WARNING: Could not find successful merge examples for test players. There might still be name mismatches in your data.")

    print("\n--- EXPLANATION: Why are some players 'Did Not Qualify' with 0 UCL Goals? ---")
    print("This happens for two main reasons, and both are CORRECT for the model:")
    print("  1. Name Mismatch During Merge: If a team's name in the league stats file (e.g., 'Inter') doesn't perfectly match the name in the UCL stats file (e.g., 'Internazionale'), the merge fails for that player. Our script tries to fix common ones, but some may remain.")
    print("  2. Genuine Non-Participation: The player's team was simply not in the Champions League that season. For example, any player from a team that finished 10th in their domestic league will correctly have 'Did Not Qualify' and 0 UCL goals.")
    
    return master_df

# --- Main Execution ---
final_master_dataset = create_and_audit_master_dataset()

if final_master_dataset is None:
    print("\nMerging process failed due to an error.")

Starting the definitive data merging process...
All four historical datasets loaded successfully.

Step 1/5: Cleaning and standardizing all data for a perfect merge...
Data cleaning and standardization complete.

Step 2/5: Verifying 'UCL_progress' column exists...
'UCL_progress' column successfully found.

Step 3/5: Merging all datasets...
All merges complete.

Step 4/5: Performing final cleanup...
Cleanup complete.

Step 5/5: Master dataset created and saved to: ../data/master_dataset_2011-2025.csv


--- MERGE AUDIT & EXPLANATION ---

To verify the merge, let's check some famous players known for UCL success:

❌ AUDIT WARNING: Could not find successful merge examples for test players. There might still be name mismatches in your data.

--- EXPLANATION: Why are some players 'Did Not Qualify' with 0 UCL Goals? ---
This happens for two main reasons, and both are CORRECT for the model:
  1. Name Mismatch During Merge: If a team's name in the league stats file (e.g., 'Inter') doesn't perfe

In [30]:
import pandas as pd
import os

def run_merge_audit():
    """
    Performs a diagnostic merge to identify why UCL progress is not matching.
    """
    print("--- Starting Merge Audit ---")
    data_path = '../data/'

    try:
        # Load the two key files for this specific problem
        df_players = pd.read_csv(os.path.join(data_path, 'combined_player_stats_2011-2025_CLEAN.csv'))
        df_ucl_teams = pd.read_csv(os.path.join(data_path, 'all_ucl_team_performance_and_progress.csv'))
        print("Files loaded successfully for audit.")

    except FileNotFoundError as e:
        print(f"ERROR: A file was not found. Missing file: {e.filename}")
        return

    # --- Clean the key columns in both DataFrames ---
    for df in [df_players, df_ucl_teams]:
        df.columns = df.columns.str.strip()
        if 'Squad' in df.columns:
            df['Squad'] = df['Squad'].str.strip()
        if 'Season' in df.columns:
            df['Season'] = df['Season'].astype(str).str.strip()

    # --- Perform an Indicator Merge ---
    # This special merge adds a '_merge' column that tells us the source of each row
    audit_df = pd.merge(
        df_players[['Player', 'Squad', 'Season']], # We only need keys from the player file
        df_ucl_teams[['Squad', 'Season', 'UCL_progress']], # And keys from the UCL file
        on=['Squad', 'Season'],
        how='left', # A 'left' merge keeps every player from the left file
        indicator=True # This is the magic part!
    )

    # --- Find and Display the Mismatches ---
    # Filter for rows that were only in the 'left' file (the player stats)
    # This means they failed to find a match in the UCL teams file
    failed_matches = audit_df[audit_df['_merge'] == 'left_only']

    if not failed_matches.empty:
        print("\n--- AUDIT RESULTS: Found Mismatches! ---")
        print("The following Squads and Seasons from your player file could NOT find a match in your UCL teams file.")
        print("This is why 'UCL_progress' is being filled with 'Did Not Qualify'.\n")
        
        # Get a unique list of the failing squad names
        failing_squads = failed_matches[['Squad', 'Season']].drop_duplicates().sort_values(by='Squad')
        
        print("Unique Squad/Season combinations that are failing to merge:")
        display(failing_squads.head(20)) # Display the first 20 mismatches
    else:
        print("\n--- AUDIT RESULTS: No Mismatches Found ---")
        print("The merge keys ('Squad', 'Season') appear to be perfectly aligned.")

# --- Run the Audit ---
run_merge_audit()

--- Starting Merge Audit ---
Files loaded successfully for audit.

--- AUDIT RESULTS: Found Mismatches! ---
The following Squads and Seasons from your player file could NOT find a match in your UCL teams file.
This is why 'UCL_progress' is being filled with 'Did Not Qualify'.

Unique Squad/Season combinations that are failing to merge:


,Squad,Season
30865,Ajaccio,2022-2023
25749,Ajaccio,2013-2014
25189,Ajaccio,2012-2013
24634,Ajaccio,2011-2012
19957,Alavés,2018-2019
18838,Alavés,2016-2017
19384,Alavés,2017-2018
23474,Alavés,2024-2025
20500,Alavés,2019-2020
21070,Alavés,2020-2021
